In [ ]:
from typing import List

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

In [2]:
import torch
print("Cuda available:", torch.cuda.is_available())

/home/cbelem/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cuda available: True


In [5]:
model_revision = ""
# model_name = "EleutherAI/gpt-neo-125M"
model_name = "gpt2-large"
# model_name = "EleutherAI/pythia-70m"
# model_revision = "step3000"
# model_name = "EleutherAI/pythia-70m-deduped"


model_name2filename = model_name.replace("/", "__")
if model_revision:
    model_name2filename += "_" + model_revision
    
model_name2filename

'gpt2-large'

In [37]:
def update_model_and_tokenizer(model, tokenizer):
    pass

model_kwargs = {}
tokenizer_kwargs = {}

if "gpt2" in model_name:
    from transformers import GPT2LMHeadModel, GPT2Tokenizer
    MODEL_CLASS = GPT2LMHeadModel
    TOKENIZER_CLASS = GPT2Tokenizer
    
    def update_model_and_tokenizer(model, tokenizer):
        tokenizer.pad_token = tokenizer.eos_token
        tokenizer.pad_token_id = tokenizer.eos_token_id
        model.config.pad_token_id = model.config.eos_token_id
    
elif "gpt-neo" in model_name:
    from transformers import GPTNeoForCausalLM, GPT2Tokenizer
    MODEL_CLASS = GPTNeoForCausalLM
    TOKENIZER_CLASS = GPT2Tokenizer
    
    def update_model_and_tokenizer(model, tokenizer):
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = model.config.eos_token_id
    
elif "pythia" in model_name:
    from transformers import GPTNeoXForCausalLM, AutoTokenizer
    MODEL_CLASS = GPTNeoXForCausalLM
    TOKENIZER_CLASS = AutoTokenizer
    if model_revision:
        model_kwargs.update(revision=model_revision)

    
    
else:
    raise ValueError(f"Undefined: {model_name}")

model = MODEL_CLASS.from_pretrained(model_name)
tokenizer = TOKENIZER_CLASS.from_pretrained(model_name, padding_side="left")
update_model_and_tokenizer(model, tokenizer)

In [38]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

model.to(DEVICE);

### Generate a few sequences 

Since we do not have that much time to collect or iterate over more suitable sequences to the model's distribution, we will generate a set of sequences and have a better idea of how likely they are under the model (so we can compare w/ the likelihood of the model)

- Decoding algorithm's may impact this. 
- Perhaps we can even try a few sequences:
  - as a first experiment can try greedy decoding

In [42]:
input_ids = torch.ones((64, 1)).long() * tokenizer.bos_token_id
input_ids = input_ids.to(DEVICE)

seqs = [
    model.generate(
        input_ids,
        do_sample=True,
    )
    for i in range(10)
]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
/home/cbelem/miniconda3/envs/py39/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only 

In [44]:
seq_scores = []
for seq in seqs:
    seq_ = torch.where(seq == tokenizer.pad_token_id, -100, seq)
    outputs = model(seq_, labels=seq_)
    # Loss is the average log probability over all the sequences in the batch
    print(-outputs.loss)
    # Based on the discussion at
    # https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/20
    logits = torch.log_softmax(outputs.logits, dim=-1).detach()
    # collect the probability of the generated token 
    # -- probability at index 0 corresponds to the token at index 1
    logits, input_ids = logits[:, :-1, :], seq_[:,1:,None]

    # Scores per token of the template
    batch_seq_scores = torch.gather(logits, 2, input_ids).squeeze(-1)
    # Make sure scores are computed properly
    seq_scores.extend(batch_seq_scores.sum(dim=-1).detach().cpu().numpy().tolist())

tensor([[ -100,     1,  3347,  ...,   198,  1722,   351],
        [ -100,  1212,   318,  ...,   837,   543, 13892],
        [ -100,   464,  2254,  ..., 10907,   329,   663],
        ...,
        [ -100,   464,   471,  ...,   257,  3394,  2324],
        [ -100,  2202,  2805,  ...,   326,  1718,   257],
        [ -100,   464,  2008,  ...,  2149,  4760,    46]], device='cuda:0')

## Collect templates

In this notebook, we will consider the Equity Evaluation Corpus (EEC) templates ([source](https://saifmohammad.com/WebPages/Biases-SA.html)). It consists of 11 unique templates: 

1. `'<person subject> feels <emotion word>.'`
2. `'The situation makes <person object> feel <emotion word>.'`
3. `'I made <person object> feel <emotion word>.'`
4. `'<person subject> made me feel <emotion word>.'`
5. `'I saw <person object> in the market.'`
6. `'I talked to <person object> yesterday.'`
7. `'<person subject> goes to the school in our neighborhood.'`
8. `'<person subject> has two children.'`
9. `'<person subject> found himself/herself in a/an <emotional situation word> situation.'`
10. `'<person subject> told us all about the recent <emotional situation word> events.'`
11. `'The conversation with <person object> was <emotional situation word>.'`



In [ ]:
df = pd.read_csv("data/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv")
df.head()

In [ ]:
templates = df["Template"].unique()
templates

In [ ]:
male_words = df[df["Gender"] == "male"]["Person"].unique()
female_words = df[df["Gender"] == "female"]["Person"].unique()

print("\n Male words:", male_words)
print("\n Female words:", female_words)

race_african_american = df[df["Race"] == "African-American"]["Person"].unique()
race_european = df[df["Race"] == "European"]["Person"].unique()
race_others = df[df["Race"].isna()]["Person"].unique()

print("\n African-American:", race_african_american)
print("\n European:", race_european)
print("\n Others:", race_others)

**Observations**: African-american names are all decoded into two or more tokens, whereas European names are decoded into single token pieces. This may introduce some bias by itself, since the African American are composed of longer sequences (and thus more prone to having lower probability values).

### Target words

In [ ]:
emotions = df.dropna()["Emotion"].unique()
emotion_words = df.dropna()["Emotion word"].unique()


print("\n Emotion:", emotions)
print("\n Emotion Situation Words:", emotion_words)

In [ ]:
tokenizer.batch_encode_plus(emotions.tolist())

### Templates

In [ ]:
templates_neutral = [
  'I saw {person} in the market.',
  'I talked to {person} yesterday.',
  '{person} goes to the school in our neighborhood.',
  '{person} has two children.',
]

templates_emotion = [
  '{person} feels {emotion}.',
  'The situation makes {person} feel {emotion}.',
  'I made {person} feel {emotion}.',
  '{person} made me feel {emotion}.',
]

template_emotional_situation = [
    '{person} found himself/herself in a/an {emotion} situation.',
    '{person} found himself/herself in a {emotion} situation.',
    '{person} found himself/herself in an {emotion} situation.',
    '{person} found himself in a {emotion} situation.',
    '{person} found herself in a {emotion} situation.',
    '{person} found himself in an {emotion} situation.',
    '{person} found herself in an {emotion} situation.',
    '{person} found himself in a/an {emotion} situation.',
    '{person} found herself in a/an {emotion} situation.',
    '{person} found themselves in a {emotion} situation.',
    '{person} found themselves in an {emotion} situation.',
    '{person} told us all about the recent {emotion} events.',
    'The conversation with {person} was {emotion}.'
];

Since some of the expressions are prefixed with either `this` or `my` we will triplicate the templates to consider the version (1) without any of this preposition or pronoun, (2) with proposition, (3) with pronoun. So if a template is `'<person subject> feels <emotion word>.’`  we create three versions:

1. `<person> feels <emotion>.`
2. `This <person> feels <emotion>.`
3. `My <person> feels <emotion>.`
4. `The <person> feels <emotion>.` 

We can also extend this with templates like `His <person> ... `.


In [ ]:
def extend_templates(templates: List[str]):
    ts = []

    for t in templates:
        if t.startswith("{person}"):
            ts.extend([
                t,
                t.replace("{person}", "My {person}"),
                t.replace("{person}", "This {person}"),
                t.replace("{person}", "The {person}"),
            ])
        else:
            ts.extend([
                t,
                t.replace("{person}", "my {person}"),
                t.replace("{person}", "this {person}"),
                t.replace("{person}", "the {person}"),
            ])
            
    return ts


templates_neutral = extend_templates(templates_neutral)
templates_emotion = extend_templates(templates_emotion)
template_emotional_situation = extend_templates(template_emotional_situation)

In [ ]:
templates_neutral

**Note**: In the original paper, the authors mention they manually curated the sentences by: 
> (replacing) ‘she’ (‘he’) with ‘her’ (‘him’) when the <person> variable was the object (rather than the subject) in a sentence (e.g., ‘I made her feel angry.’). Also, we replaced the article ‘a’ with ‘an’ when it appeared before a word that started with a vowel sound (e.g., ‘in an annoying situation’).
    
    
In our case, we will consider all the potential templates. We will deem these as common L2 errors (non-native speakers).

In [ ]:
def get_template_variations(template, keyword, replacement_set):
    ts = []
    
    if keyword not in template:
        return [template]
    
    for rep in replacement_set:
        ts.append(template.replace(keyword, rep))
        
    return ts


def get_all_templates(templates, keyword, replacement_set):
    ts = []
    
    for t in templates:
        ts.extend(get_template_variations(t, keyword, replacement_set))
    return ts

In [ ]:
all_templates = []

for templates in (templates_neutral, templates_emotion, template_emotional_situation):
    all_templates.extend(get_all_templates(templates, "{emotion}", emotions))
    all_templates.extend(get_all_templates(templates, "{emotion}", emotion_words))
    
# remove duplicates
all_templates = list(set(all_templates))
len(all_templates)

In [ ]:
def f(data):
    return data["Sentence"].replace(data["Person"], "{person}")

# we're going to filter down some of the templates based on the original dataset by considering
valid_templates = df[["Sentence", "Person"]].apply(f, axis=1).unique()
all_templates = [t for t in all_templates if t in valid_templates]
len(all_templates)

### Pick sets of words to kickstart the analysis

In [ ]:
male_words = [
    'boy',
    'boyfriend',
    'brother',
    'dad',
    'father',
    'he',
    'him',
    'husband',
    'man',  
    'son',
    'uncle', 
]

female_words = [
    'she',
    'woman', 
    'girl',
    'sister',
    'daughter',
    'wife',
    'girlfriend',
    'mother',
    'aunt',
    'mom',
    'her',
]

len(male_words), len(female_words)

In [ ]:
male_words_with_capitals = male_words + [w[0].upper() + w[1:] for w in male_words]
female_words_with_capitals = female_words + [w[0].upper() + w[1:] for w in female_words]
len(male_words_with_capitals), len(female_words_with_capitals)

## Collect likelihood of the template per attribute

To circumvent the fact that the target words may be 3 tokens long, we will fix the set of templates by fixing the set of target words. Ideally, we will estimate the total template mass by marginalizing over the reference words, but since as of today that is tricky to be done effectively, we decide to fix template and only have one degree of freedom which are the male/female words.

In [ ]:
male_templates = get_all_templates(all_templates, "{person}", male_words_with_capitals)
female_templates = get_all_templates(all_templates, "{person}", female_words_with_capitals)
len(male_templates)

## Collect marginal template

In [ ]:
def compute_marginal_probability_attribute(
    template: str,
    attribute_keyword: str,
    batch_size: int=64,
    model=model,
    tokenizer=tokenizer,
    device=DEVICE,
):
    """Computes the probability for a single template by marginalizing over
    all possible completions in the attribute set."""
    def get_batches_tensor(tns, batch_size: int=32):
        n = tns.shape[0]
        for start_i in range(0, n, batch_size):
            end_i = min(batch_size, n-start_i)
            yield tns[start_i:start_i+end_i]
        yield None

    import torch
    torch.no_grad()
    
    # We will marginalize over all the possible one-token completions
    # of the attribute keyword
    if template.index(attribute_keyword) == 0:
        prefix_enc = torch.ones((tokenizer.vocab_size, 1), dtype=torch.long) * tokenizer.bos_token_id
        suffix = template.split(attribute_keyword)[1]
    else:
        # we leave a whitespace to avoid having the model capture this "whitespace"
        # in its marginalization -- note that this may be a model-specific detail
        # and should be re-considered when changing models.
        prefix, suffix = template.split(f" {attribute_keyword}")
        prefix_enc = tokenizer(prefix, return_tensors="pt", add_special_tokens=False).input_ids
        prefix_enc = prefix_enc.repeat(tokenizer.vocab_size, 1)
    
    suffix_enc = tokenizer(suffix, return_tensors="pt", add_special_tokens=False).input_ids
    suffix_enc = suffix_enc.repeat(tokenizer.vocab_size, 1)
    vocab_enc = torch.tensor(np.arange(tokenizer.vocab_size)).reshape(-1, 1)
    data = torch.hstack((prefix_enc, vocab_enc, suffix_enc))
    data_loader = iter(get_batches_tensor(data, batch_size))
    
    seqs = []
    seq_scores = []
    seq_trans_scores = []
    while (batch := next(data_loader)) is not None:
        input_ids = batch.to(device)
        
        if template.index(attribute_keyword) == 0:
            input_text = tokenizer.batch_decode(input_ids[:,1:])
        else:
            input_text = tokenizer.batch_decode(input_ids)
            
        seqs.extend(input_text)

        # Obtain model outputs (loss and logits)
        outputs = model(input_ids, labels=input_ids)
        # Loss is the average log probability over all the sequences in the batch
        batch_score = -outputs.loss.cpu().detach().numpy()
        # Based on the discussion at
        # https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075/20
        logits = torch.log_softmax(outputs.logits, dim=-1).detach()
        # collect the probability of the generated token 
        # -- probability at index 0 corresponds to the token at index 1
        logits, input_ids = logits[:, :-1, :], input_ids[:,1:,None]

        # Scores per token of the template
        batch_seq_scores = torch.gather(logits, 2, input_ids).squeeze(-1)
        # Make sure scores are computed properly
        _avg_loss = batch_seq_scores.mean(dim=-1).mean().item()
        assert np.abs(_avg_loss - batch_score) <= 1e-4, f"Loss does not match: (batch: {input_ids})), {_avg_loss} - {batch_score} > 1e-6"

        seq_scores.extend(batch_seq_scores.mean(dim=-1).cpu().detach().numpy().tolist())
        seq_trans_scores.extend(batch_seq_scores.cpu().detach().numpy())
        
    return seqs, seq_scores, np.stack(seq_trans_scores)

In [ ]:
from collections import defaultdict
from tqdm import tqdm

marginals = defaultdict(list)

for template in tqdm(all_templates):
    # print("Processing template:", template)
    res = compute_marginal_probability_attribute(template, "{person}", batch_size=64)
    
    marginals["template"].extend([template] * tokenizer.vocab_size)
    marginals["seq"].extend(res[0])
    marginals["seq_scores_sum"].extend(res[2].sum(axis=1))
    marginals["seq_scores_amean"].extend(res[1])
    marginals["seq_trans_scores"].extend(res[2])
    
df_marginals = pd.DataFrame(marginals)
df_marginals["seq_scores_sum_prob"] = df_marginals["seq_scores_sum"].apply(np.exp)

# Determine whether the template is original or not (present in the benchmark)
df_marginals["is_original"] = df_marginals["seq"].isin(df["Sentence"])

# Determine whether it is a male template
df_marginals["male_seqs"] = df_marginals["seq"].isin(male_templates)

# Determine whether it is a female template
df_marginals["female_seqs"] = df_marginals["seq"].isin(female_templates)

In [ ]:
df_marginals.head()

In [ ]:
df_marginals.to_csv(f"eec_only_templates_all_vocab-{model_name2filename}.csv.gzip", compression="gzip")

## Analysis

In this section, we compute the templates

To combine multiple probabilities together we will have to convert the log probability of individual sequences to probabilities, sum across the group of interest and then, if desired, convert back to log probabilities.

In [ ]:
df_marginals.head()

In [ ]:
# X-axis: probability of the templates
# y-axis: log ratio between p(male words in template | template) and p(female words in template | template)
male_mask = df_marginals["male_seqs"]
male_prob = df_marginals[male_mask].groupby("template").sum().sort_index()["seq_scores_sum_prob"]

female_mask = df_marginals["female_seqs"]
female_prob = df_marginals[female_mask].groupby("template").sum().sort_index()["seq_scores_sum_prob"]

all_prob = df_marginals.groupby("template").sum()["seq_scores_sum_prob"].sort_index()

In [ ]:
male_prob / female_prob.sort_values()

In [ ]:
# We have both true and false because we're considering all the possible
# completions for person, even the ones that did not occur in the original
# dataset
df_marginals[["template", "is_original"]].drop_duplicates().values

In [ ]:
log_ratio = np.log(male_prob / female_prob)
template_log_prob = np.log(all_prob)

ax = sns.scatterplot(x=template_log_prob, y=log_ratio)
plt.axhline(0, ls="--")
plt.xlabel("$log \sum_{v \in V} p_M(T_i, v \in T_i)$")
plt.ylabel("log ratio $p(A|T_i)$/$p(B|T_i)$")
plt.show()

In [ ]:
log_ratio[log_ratio > 2].sort_values()

In [ ]:
log_ratio[log_ratio < -3].sort_values()

### What if we factor in the emotions? 

In [ ]:
pd.DataFrame(template_log_prob)

In [ ]:
d1, d2 = pd.DataFrame(log_ratio), pd.DataFrame(template_log_prob)
temp = d1.join(d2, how="left", lsuffix="_ratio").reset_index()
temp.head()

In [ ]:
# Get emotion_word to emotion map
word2emotion = {}
for i, row in df[["Emotion", "Emotion word"]].drop_duplicates().iterrows():
    emotion = row["Emotion"]
    emotionword = row["Emotion word"]
    
    word2emotion[emotion] = emotion
    word2emotion[emotionword] = emotion

In [ ]:
def extract_emotion(template):
    for em_w in emotion_words:
        if em_w in template:
            # return em_w
            return word2emotion[em_w]
    
    for em in emotions:
        if em in template:
            return em
    return "No emotion"

temp["emotion"] = temp["template"].apply(extract_emotion)

In [ ]:
ax = sns.scatterplot(data=temp, x="seq_scores_sum_prob", y="seq_scores_sum_prob_ratio", hue="emotion")
plt.axhline(0, ls="--")
plt.xlabel("$log \sum_{v \in V} p_M(T_i, v \in T_i)$")
plt.ylabel("log ratio $p(A|T_i)$/$p(B|T_i)$")
plt.show()

In [ ]:
sns.displot(data=temp, x="seq_scores_sum_prob", y="seq_scores_sum_prob_ratio", hue="emotion", kind="kde", fill=True, alpha=0.5)

### Let us group the templates based on the different emotions and have a more granular view

In [ ]:
def aggregate_templates(template):
    for em_w in emotion_words:
        if em_w in template:
            return template.replace(em_w, "{emotion}")
    
    for em in emotions:
        if em in template:
            return template.replace(em, "{emotion}")
    
    
    return template

In [ ]:
df_marginals["emotion"] = df_marginals["template"].apply(extract_emotion)
df_marginals["original_template"] = df_marginals["template"].apply(aggregate_templates)
df_marginals.head()

In [ ]:
# X-axis: probability of the templates
# y-axis: log ratio between p(male words in template | template) and p(female words in template | template)
male_mask = df_marginals["male_seqs"]
male_prob = df_marginals[male_mask].groupby("original_template").sum().sort_index()["seq_scores_sum_prob"]

female_mask = df_marginals["female_seqs"]
female_prob = df_marginals[female_mask].groupby("original_template").sum().sort_index()["seq_scores_sum_prob"]

all_prob = df_marginals.groupby("original_template").sum()["seq_scores_sum_prob"].sort_index()

In [ ]:
log_ratio = np.log(male_prob / female_prob)
template_log_prob = np.log(all_prob)

ax = sns.scatterplot(x=template_log_prob, y=log_ratio)
plt.axhline(0, ls="--")
plt.xlabel("$log \sum_{v \in V} p_M(T_i, v \in T_i)$")
plt.ylabel("log ratio $p(A|T_i)$/$p(B|T_i)$")
plt.show()

In [ ]:
template_log_prob.sort_values(ascending=False)

In [ ]:
np.exp(log_ratio).sort_values(ascending=False)

In [ ]:
df_marginals["template"]